## ! pip install -q tensorflow numpy
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./
! cp -r drive/My\ Drive/Colab\ Notebooks/models/ensemble ./

In [1]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [2]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [3]:
def ds_from_df(df, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Meme')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [4]:
from tensorflow import keras

alex = keras.models.load_model('ensemble/alexnet_ffwd')
dense = keras.models.load_model('ensemble/densenet_ffwd')
resnet = keras.models.load_model('ensemble/resnet_ffwd')

OSError: No file or directory found at ensemble/alexnet_ffwd

In [ ]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

In [ ]:
import numpy as np
import pickle

des_X, alex_X, res_X, y = pickle.load(open('densenest_embeddings.pkl', 'rb')), pickle.load(open('alexnet_embeddings.pkl', 'rb')), np.array([embedding[1][i].split() for i in range(1600)]).astype(float), df[['Meme']].values

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

ssc = StandardScaler()
mms = MinMaxScaler()

In [ ]:
import pickle
from datetime import date

temp = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ssc.fit_transform(df[['Engagement']].values)
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ohe(create_map())
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = pickle.load(open('cls_umberto_embeddings.pkl', 'rb'))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

In [ ]:
from sklearn.model_selection import train_test_split

res_X_train, res_X_test, res_y_train, res_y_test = train_test_split(res_X, y, test_size=0.2, stratify=y, random_state=42)
des_X_train, des_X_test, des_y_train, des_y_test = train_test_split(des_X, y, test_size=0.2, stratify=y, random_state=42)
alex_X_train, alex_X_test, alex_y_train, alex_y_test = train_test_split(alex_X, y, test_size=0.2, stratify=y, random_state=42)

In [ ]:
buffer_size = 10000
batch_size = 64
num_epochs = 10

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

ds_res = tf.data.Dataset.from_tensor_slices((res_X_test, res_y_test))
ds_res = ds_res.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_res = ds_res.batch(batch_size)

ds_des = tf.data.Dataset.from_tensor_slices((des_X_test, des_y_test))
ds_des = ds_des.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_des = ds_des.batch(batch_size)

ds_alex = tf.data.Dataset.from_tensor_slices((alex_X_test, alex_y_test))
ds_alex = ds_alex.shuffle(buffer_size=buffer_size,
                            reshuffle_each_iteration=False)
ds_alex = ds_alex.batch(batch_size)

In [ ]:
# del a, d, r

a = alex.fit(ds_alex,
             epochs=num_epochs,
             batch_size=batch_size,
             use_multiprocessing=True)

d = dense.fit(ds_des,
              epochs=num_epochs,
              batch_size=batch_size,
              use_multiprocessing=True)
              
r = resnet.fit(ds_res,
               epochs=num_epochs,
               batch_size=batch_size,
               use_multiprocessing=True)

In [ ]:
alex.save('models/ensemble_eval/alexnet_ffwd')
dense.save('models/ensemble_eval/densenet_ffwd')
resnet.save('models/ensemble_eval/resnet_ffwd')

In [ ]:
! cp -r models ./drive/My\ Drive/Colab\ Notebooks/